In [ ]:
include("table.jl")

In [ ]:
results = load_results()

In [ ]:
ENV["COLUMNS"] = 200

In [ ]:
df = results2df(results)

In [ ]:
sort(df[(df.model .== "EQ") .& (df.train_mat .== "COV"),:], [:train_d, :train_gtype, :train_mat])

In [ ]:
sort(df[(df.model .== "EQ") .& (df.train_mat .== "COR"),:], [:train_d, :train_gtype, :train_mat])

In [ ]:
sort(df[df.model .== "CNN",:], [:train_d, :train_gtype, :train_mat])

# Test dataframe selection

And exploring the data

In [ ]:
df[1:4,["model","train_d"]]

In [ ]:
# sum(df.raw)

# I don't need train_k, because all of them are 1s
df = df[!, Not(All(:train_k, :train_noise))]

unique(df.train_mat)
unique(df.model)

In [ ]:
function select_sep_df(df, model)
    sort(df[df.model .== model,
                    # test_d is same as train_d
                    Not(All(:train_k, :train_noise, :test_d))],
                 [:train_mec, :train_gtype, :test_gtype, :train_d, :test_k])
end

function select_ensemble_df(df, model)
    subdf = sort(df[df.model .== model, Not(All(:train_k, :train_noise, r"train_"))],
                 [:test_gtype, :test_d, :test_k])
    show(subdf, allrows=true)
    nothing
end

In [ ]:
# 1. all the models in separate mode
select_sep_df(df, "deep-FC")

In [ ]:
select_sep_df(df, "deep-EQ")

# Pre-defined tables

In [ ]:
table_cmp_baseline(df)

In [ ]:
table_ER24(df)

In [ ]:
table_raw(df)

In [ ]:
table_transfer_k(df)

In [ ]:
table_transfer_noise(df)

In [ ]:
table_transfer_gtype(df)

In [ ]:
table_ensemble(df)

# Testing displaysize

In [ ]:
displaysize()

In [ ]:
# show all of df
show(df[1:3,:], allcols=true)

In [ ]:
IOContext(stdout, :displaysize=>(30,100)) |> displaysize